In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
import keras_ocr

In [ ]:
# Load the pre-trained model
pipeline = keras_ocr.pipeline.Pipeline() 

In [14]:
 # Provide an image for OCR
image_path = './02.png'
image = keras_ocr.tools.read(image_path)


In [15]:
# Perform OCR
predictions = pipeline.recognize([image])

1/1 [==============================] - 7s 7s/step


2023-05-30 21:48:03.908191: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-30 21:48:03.910585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-30 21:48:03.912392: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 3s 3s/step


In [16]:
# # Extract the predictions and left coordinates
# prediction_boxes = predictions[0]['boxes']
# left_coordinates = [box[0][0] for box in prediction_boxes]

TypeError: list indices must be integers or slices, not str

In [17]:
# # Extract the predictions and left coordinates
# prediction_boxes = predictions[0][0]['boxes']
# left_coordinates = [box[0] for box in prediction_boxes]

TypeError: tuple indices must be integers or slices, not str